In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

In [ ]:
PATH_INPUT = "../input/vinbigdata-chest-xray-abnormalities-detection/"
df_labels=pd.read_csv(PATH_INPUT+"train.csv")
df_labels["path"]=df_labels.image_id.apply(lambda x: PATH_INPUT+f"train/{x}.dicom")
df_labels.head()

In [ ]:
df_labels.class_name.value_counts()

In [ ]:
df_labels.image_id.nunique()

In [ ]:
# Number of regions per image
df_labels.query("class_id!=14").image_id.value_counts().plot.hist(bins=50,log=True)

In [ ]:
# Colormap
labels_info=df_labels.query("class_id!=14")[["class_id","class_name"]].drop_duplicates().sort_values(by="class_id").reset_index(drop=True)
labels_info["color"]=sns.color_palette("husl", labels.shape[0])
labels_info["color"]=labels_info["color"].apply(lambda x: tuple([int(cl*255) for cl in x]))
labels_info

In [ ]:
## Reading a dicom file
# From: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage import exposure
import cv2

def dicom2array(path, voi_lut=True, fix_monochrome=True, equalize_hist=False):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
        
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    
    if equalize_hist:
        data = exposure.equalize_hist(data)
        
    return data

def plot_img(img, size=(7, 7), is_rgb=True, title="", cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()
    
def plot_img_with_labels(image_id,df_labels, labels_info, size=(7, 7), is_rgb=True, title="", cmap='gray'):
    df_plot = df_labels.query(f"image_id=='{image_id}'")
    img=dicom2array(df_plot.iloc[0].path)
    
    # To rgb:
    img=cv2.merge((img, img, img))
    # Draw boxes if present
    df_boxes = df_plot.query("class_id!=14")
    if df_boxes.shape[0]>0:
        for i,row in df_boxes.iterrows(): #x_min	y_min	x_max	y_max
            img = cv2.rectangle(img,(int(row.x_min), int(row.y_min)),(int(row.x_max), int(row.y_max)), labels_info.loc[row.class_id]["color"],5)
            
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

In [ ]:
df_labels.query("image_id=='9a5094b2563a1ef3ff50dc5c7ff71345'")

In [ ]:
import random

for img_id in random.choices(df_labels.image_id.unique(),k=10):
    plot_img_with_labels(img_id,df_labels, labels_info)

In [ ]:
path_example=df_labels.loc[0].path
print(path_example)
dicom2array(path_example)

In [ ]:
# Generate images in png format
imgs_to_process=df_labels.image_id.unique()

input_paths = [f"../input/vinbigdata-chest-xray-abnormalities-detection/train/{x}.dicom" for x in paths_to_process] 
os.mkdir("./preprocessed/")
output_paths= [f"./preprocessed/{x}.png" for x in imgs_to_process]


for inp,out in tqdm(zip(input_paths,output_paths)):
    

In [ ]:
paths_to_process[0]

In [ ]:
%%bash
git clone https://github.com/ahmedfgad/Mask-RCNN-TF2.git
mv -f Mask-RCNN-TF2/* .
rm -rf Mask-RCNN-TF2

In [ ]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

ROOT_DIR="./"
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
class XrayConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "xray"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + df_labels.shape[0]  # background + labels

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 1000

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 500
    
config = XrayConfig()
config.display()